In [1]:
import numpy as np

# Define parameters (example values, replace with actual parameters)
T = 3  # number of time slots
K = 4  # number of users
A_bar = np.array([1.0, 0.9, 1.1, 1.2])  # Example values for A_bar
B_bar = np.array([0.1, 0.1, 0.2, 0.1])  # Example values for B_bar
B_tilde = np.array([[0.0, 0.05, 0.02, 0.01],
                    [0.05, 0.0, 0.03, 0.02],
                    [0.02, 0.03, 0.0, 0.04],
                    [0.01, 0.02, 0.04, 0.0]])  # Example values for B_tilde
I_M = np.array([0.1, 0.2, 0.1, 0.15])  # Example values for I_M
b_t = np.random.rand(T, K)  # Random values for b_t (replace with actual values)

# Function to check feasibility of power allocations
def check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, b_t, T, K):
    for t in range(T):
        p_t = np.zeros(K)
        gamma_t = np.zeros(K)
        for j in range(K):
            gamma_t[j] = 2 ** (eta_t * b_t[t, j]) - 1
        for j in range(K):
            numerator = gamma_t[j] * (np.sum(p_t * B_tilde[j]) + I_M[j])
            denominator = A_bar[j] - gamma_t[j] * B_bar[j]
            if denominator <= 0:
                return False
            p_t[j] = numerator / denominator
            if p_t[j] < 0 or p_t[j] > 1:
                return False
    return True

# Iteratively find maximum eta_t
def maximize_eta_t(A_bar, B_bar, B_tilde, I_M, b_t, T, K, eta_t_start=0.1, eta_t_step=0.01, max_iterations=1000):
    eta_t = eta_t_start
    for _ in range(max_iterations):
        if check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, b_t, T, K):
            eta_t += eta_t_step
        else:
            eta_t -= eta_t_step
            break
    return eta_t

# Example usage
max_eta_t = maximize_eta_t(A_bar, B_bar, B_tilde, I_M, b_t, T, K)
print("Maximum eta_t:", max_eta_t)


C:\Users\afmb\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\afmb\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\afmb\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Maximum eta_t: 2.1299999999999986


In [2]:
import numpy as np

# Define parameters (example values, replace with actual parameters)
T = 3  # number of time slots
K = 4  # number of users
A_bar = np.array([1.0, 0.9, 1.1, 1.2])  # Example values for A_bar
B_bar = np.array([0.1, 0.1, 0.2, 0.1])  # Example values for B_bar
B_tilde = np.array([[0.0, 0.05, 0.02, 0.01],
                    [0.05, 0.0, 0.03, 0.02],
                    [0.02, 0.03, 0.0, 0.04],
                    [0.01, 0.02, 0.04, 0.0]])  # Example values for B_tilde
I_M = np.array([0.1, 0.2, 0.1, 0.15])  # Example values for I_M
b_t = np.random.rand(T, K)  # Random values for b_t (replace with actual values)

# Function to check feasibility of power allocations and return power values
def check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, b_t, T, K):
    powers = []
    for t in range(T):
        p_t = np.zeros(K)
        gamma_t = np.zeros(K)
        for j in range(K):
            gamma_t[j] = 2 ** (eta_t * b_t[t, j]) - 1
        for j in range(K):
            numerator = gamma_t[j] * (np.sum(p_t * B_tilde[j]) + I_M[j])
            denominator = A_bar[j] - gamma_t[j] * B_bar[j]
            if denominator <= 0:
                return False, None
            p_t[j] = numerator / denominator
            if p_t[j] < 0 or p_t[j] > 1:
                return False, None
        powers.append(p_t)
    return True, powers

# Iteratively find maximum eta_t and store power allocations
def maximize_eta_t(A_bar, B_bar, B_tilde, I_M, b_t, T, K, eta_t_start=0.001, eta_t_step=0.1, max_iterations=1000):
    eta_t = eta_t_start
    best_eta_t = eta_t
    best_powers = None
    for _ in range(max_iterations):
        feasible, powers = check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, b_t, T, K)
        if feasible:
            best_eta_t = eta_t
            best_powers = powers
            eta_t += eta_t_step
        else:
            break
    return best_eta_t, best_powers

# Example usage
max_eta_t, optimal_powers = maximize_eta_t(A_bar, B_bar, B_tilde, I_M, b_t, T, K)
print("Maximum eta_t:", max_eta_t)
print("Optimal power allocations:")
for t in range(T):
    print(f"Time slot {t + 1}: {optimal_powers[t]}")


Maximum eta_t: 2.201000000000001
Optimal power allocations:
Time slot 1: [0.27467488 0.032932   0.73955971 0.46446772]
Time slot 2: [0.42282353 0.22812505 0.92497117 0.33298435]
Time slot 3: [0.01236299 0.00965923 0.08691739 0.27843299]


In [3]:
import numpy as np
from scipy.optimize import linprog

# Define parameters (example values, replace with actual parameters)
T = 3  # number of time slots
K = 4  # number of users
A_bar = np.array([1.0, 0.9, 1.1, 1.2])  # Example values for A_bar
B_bar = np.array([0.1, 0.1, 0.2, 0.1])  # Example values for B_bar
B_tilde = np.array([[0.0, 0.05, 0.02, 0.01],
                    [0.05, 0.0, 0.03, 0.02],
                    [0.02, 0.03, 0.0, 0.04],
                    [0.01, 0.02, 0.04, 0.0]])  # Example values for B_tilde
I_M = np.array([0.1, 0.2, 0.1, 0.15])  # Example values for I_M
b_t = np.random.rand(T, K)  # Random values for b_t (replace with actual values)

# Function to construct the linear programming problem
def construct_lp_problem(eta_t, A_bar, B_bar, B_tilde, I_M, b_t, T, K):
    c = np.zeros(T * K)  # Objective function coefficients (zeros since we're not optimizing p directly)
    bounds = [(0, 1)] * (T * K)  # Bounds for each p_t^j
    
    # Constraints
    A = []
    b = []
    
    for t in range(T):
        for j in range(K):
            gamma_t_j = 2 ** (eta_t * b_t[t, j]) - 1
            row = np.zeros(T * K)
            row[t * K + j] = A_bar[j] - gamma_t_j * B_bar[j]
            for j_prime in range(K):
                if j_prime != j:
                    row[t * K + j_prime] = -gamma_t_j * B_tilde[j, j_prime]
            A.append(row)
            b.append(gamma_t_j * I_M[j])
    
    return np.array(A), np.array(b), c, bounds

# Function to maximize eta_t using linear programming
def maximize_eta_t_lp(A_bar, B_bar, B_tilde, I_M, b_t, T, K, eta_t_start=0.1, eta_t_step=0.01, max_iterations=1000):
    eta_t = eta_t_start
    best_eta_t = eta_t
    best_powers = None
    for _ in range(max_iterations):
        A, b, c, bounds = construct_lp_problem(eta_t, A_bar, B_bar, B_tilde, I_M, b_t, T, K)
        res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
        if res.success:
            best_eta_t = eta_t
            best_powers = res.x
            eta_t += eta_t_step
        else:
            break
    return best_eta_t, best_powers

# Example usage
max_eta_t_lp, optimal_powers_lp = maximize_eta_t_lp(A_bar, B_bar, B_tilde, I_M, b_t, T, K)
print("Maximum eta_t (LP):", max_eta_t_lp)
print("Optimal power allocations (LP):")
for t in range(T):
    print(f"Time slot {t + 1}: {optimal_powers_lp[t * K:(t + 1) * K]}")

# Comparison with analytical method
def maximize_eta_t_analytical(A_bar, B_bar, B_tilde, I_M, b_t, T, K, eta_t_start=0.1, eta_t_step=0.01, max_iterations=1000):
    eta_t = eta_t_start
    best_eta_t = eta_t
    best_powers = None
    for _ in range(max_iterations):
        feasible, powers = check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, b_t, T, K)
        if feasible:
            best_eta_t = eta_t
            best_powers = powers
            eta_t += eta_t_step
        else:
            break
    return best_eta_t, best_powers

max_eta_t_analytical, optimal_powers_analytical = maximize_eta_t_analytical(A_bar, B_bar, B_tilde, I_M, b_t, T, K)
print("Maximum eta_t (Analytical):", max_eta_t_analytical)
print("Optimal power allocations (Analytical):")
for t in range(T):
    print(f"Time slot {t + 1}: {optimal_powers_analytical[t]}")


Maximum eta_t (LP): 10.08999999999983
Optimal power allocations (LP):
Time slot 1: [1. 1. 0. 0.]
Time slot 2: [0. 0. 0. 0.]
Time slot 3: [0. 0. 0. 0.]
Maximum eta_t (Analytical): 2.6899999999999866
Optimal power allocations (Analytical):
Time slot 1: [0.23356001 0.19112289 0.01657727 0.06473388]
Time slot 2: [0.13565495 0.03178939 0.66838252 0.40092387]
Time slot 3: [0.07882212 0.99646468 0.00728141 0.25841178]


In [4]:
import numpy as np

# Define parameters
T = 1  # single time slot for simplicity
K = 5  # number of users
B_tilde = np.array([[0.0, 0.00280774, 0.00227045, 0.00599378, 0.00054239],
                    [0.00280774, 0.0, 0.00090963, 0.10872945, 0.00079498],
                    [0.00227045, 0.00090963, 0.0, 0.00180449, 0.03946987],
                    [0.00599378, 0.10872945, 0.00180449, 0.0, 0.00094696],
                    [0.00054239, 0.00079498, 0.03946987, 0.00094696, 0.0]])
A_bar = 2 * np.array([0.038254, 0.37979634, 6.42646204, 1.4801049, 1.43769989])
B_bar = np.array([0.00206423, 0.04449928, 1.3862424, 0.29103015, 0.29611187])
I_M = np.array([7.78643045e-13, 2.45343970e-12, 1.00922029e-11, 4.84335440e-12, 4.77346915e-12])
beta = np.array([8, 6, 8, 2, 7])

# Function to check feasibility of power allocations and return power values
def check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, beta, T, K):
    powers = []
    for t in range(T):
        p_t = np.zeros(K)
        gamma_t = np.zeros(K)
        for j in range(K):
            gamma_t[j] = 2 ** (eta_t * beta[j]) - 1
        for j in range(K):
            numerator = gamma_t[j] * (np.sum(p_t * B_tilde[j]) + I_M[j])
            denominator = A_bar[j] - gamma_t[j] * B_bar[j]
            if denominator <= 0:
                return False, None
            p_t[j] = numerator / denominator
            if p_t[j] < 0 or p_t[j] > 1:
                return False, None
        powers.append(p_t)
    return True, powers

# Bisection method to find maximum eta_t and store power allocations
def bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, T, K, eta_t_lower=0.0, eta_t_upper=10.0, tolerance=1e-6):
    best_eta_t = eta_t_lower
    best_powers = None
    while eta_t_upper - eta_t_lower > tolerance:
        eta_t_mid = (eta_t_lower + eta_t_upper) / 2.0
        feasible, powers = check_feasibility(eta_t_mid, A_bar, B_bar, B_tilde, I_M, beta, T, K)
        if feasible:
            best_eta_t = eta_t_mid
            best_powers = powers
            eta_t_lower = eta_t_mid
        else:
            eta_t_upper = eta_t_mid
    return best_eta_t, best_powers

# Example usage
max_eta_t, optimal_powers = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, T, K)
print("Maximum eta_t:", max_eta_t)
print("Optimal power allocations:")
for t in range(T):
    print(f"Time slot {t + 1}: {optimal_powers[t]}")


Maximum eta_t: 0.42007625102996826
Optimal power allocations:
Time slot 1: [1.25841241e-10 2.42313463e-11 3.02162453e-06 1.58055715e-09
 8.86540154e-07]


In [5]:
import numpy as np

# Define parameters
K = 5  # number of users
T = 1  # number of time slots, we assume one slot for simplicity
B_tilde = np.array([[0.0, 0.00280774, 0.00227045, 0.00599378, 0.00054239],
                    [0.00280774, 0.0, 0.00090963, 0.10872945, 0.00079498],
                    [0.00227045, 0.00090963, 0.0, 0.00180449, 0.03946987],
                    [0.00599378, 0.10872945, 0.00180449, 0.0, 0.00094696],
                    [0.00054239, 0.00079498, 0.03946987, 0.00094696, 0.0]])
A_bar = 2 * np.array([0.038254, 0.37979634, 6.42646204, 1.4801049, 1.43769989])
B_bar = np.array([0.00206423, 0.04449928, 1.3862424, 0.29103015, 0.29611187])
I_M = np.array([7.78643045e-13, 2.45343970e-12, 1.00922029e-11, 4.84335440e-12, 4.77346915e-12])
beta = np.array([8, 6, 8, 2, 7])  # values for beta

# Function to check feasibility of power allocations and return power values
def check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, beta, K):
    powers = 0.2*np.ones(K)
    gamma_t = 2 ** (eta_t * beta) - 1
    for j in range(K):
        numerator = gamma_t[j] * (np.sum(powers * B_tilde[j]) + I_M[j])
        denominator = A_bar[j] - gamma_t[j] * B_bar[j]
        if denominator <= 0:
            return False, None
        powers[j] = numerator / denominator
        if powers[j] < 0 or powers[j] > 1:
            return False, None
    return True, powers

# Bisection method to find maximum eta_t and store power allocations
def bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, K, eta_t_lower=0.0, eta_t_upper=10.0, tolerance=1e-6):
    best_eta_t = eta_t_lower
    best_powers = None
    while eta_t_upper - eta_t_lower > tolerance:
        eta_t_mid = (eta_t_lower + eta_t_upper) / 2.0
        feasible, powers = check_feasibility(eta_t_mid, A_bar, B_bar, B_tilde, I_M, beta, K)
        if feasible:
            best_eta_t = eta_t_mid
            best_powers = powers
            eta_t_lower = eta_t_mid
        else:
            eta_t_upper = eta_t_mid
    return best_eta_t, best_powers

# Example usage
max_eta_t, optimal_powers = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, K)
print("Maximum eta_t:", max_eta_t)
print("Optimal power allocations:", optimal_powers)


Maximum eta_t: 0.4189908504486084
Optimal power allocations: [0.37209255 0.19819429 0.99992324 0.00743238 0.29043525]


# The following works well. It is linear programming and bisection

#This is with while over linear programming:

In [29]:
import numpy as np
import math
import numpy.linalg as nl
# Define parameters
T = 1  # single time slot for simplicity
K = 5  # number of users
B_tilde = np.array([[0.0, 0.00280774, 0.00227045, 0.00599378, 0.00054239],
                    [0.00280774, 0.0, 0.00090963, 0.10872945, 0.00079498],
                    [0.00227045, 0.00090963, 0.0, 0.00180449, 0.03946987],
                    [0.00599378, 0.10872945, 0.00180449, 0.0, 0.00094696],
                    [0.00054239, 0.00079498, 0.03946987, 0.00094696, 0.0]])
A_bar = 1 * np.array([0.038254, 0.37979634, 6.42646204, 1.4801049, 1.43769989])
B_bar = np.array([0.00206423, 0.04449928, 1.3862424, 0.29103015, 0.29611187])
I_M = np.array([7.78643045e-13, 2.45343970e-12, 1.00922029e-11, 4.84335440e-12, 4.77346915e-12])
beta = np.array([8, 6, 8, 2, 7]) #the bits
eps = 0.6
# Function to check feasibility of power allocations and return power values
def check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, beta, T, K):
    itr = 0
    powers = (1/K)*np.ones((T, K))
    powers_old = 10*(1/K)*np.ones((T, K))
    for t in range(T):
        gamma_t = 2 ** (eta_t * beta) - 1
        # Solve the system of linear equations for power allocations
        while(np.linalg.norm(powers-powers_old, ord=np.inf) > eps):
            itr += 1
            print('iteration of LP:', itr)
            powers_old = np.copy(powers)
            for j in range(K):
                numerator = gamma_t[j] * (np.sum(powers[t] * B_tilde[j]) + I_M[j])
                denominator = A_bar[j] - gamma_t[j] * B_bar[j]
                if denominator <= 0:
                    return False, None
                powers[t, j] = numerator / denominator
        # Check feasibility of the computed powers
        if np.any(powers[t] < 0) or np.any(powers[t] > 1):
            return False, None
        #print('iteration of LP:', itr)
    return True, powers

# Function to compute SINR_t^j given powers
def compute_SINR(eta_t, powers, A_bar, B_bar, B_tilde, I_M, beta):
    T, K = powers.shape
    SINR = np.zeros((T, K))
    for t in range(T):
        gamma_t = 2 ** (eta_t * beta) - 1
        for j in range(K):
            interference = np.sum(powers[t] * B_tilde[j]) + I_M[j]
            SINR[t, j] = A_bar[j] * powers[t, j] / (B_bar[j] * powers[t, j] + interference)
    return SINR

# Function to compute latencies ell_t^j given SINR_t^j
def compute_latencies(SINR, beta):
    T, K = SINR.shape
    latencies = np.zeros((T, K))
    for t in range(T):
        for j in range(K):
            if SINR[t, j] > 0:
                latencies[t, j] = beta[j] / math.log2(1 + SINR[t, j])
            else:
                latencies[t, j] = float('inf')  # Handle cases where SINR[t, j] is zero
    return latencies

# Bisection method to find maximum eta_t and store power allocations
def bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, T, K, eta_t_lower=0.0, eta_t_upper=10.0, tolerance=1e-6):
    itr_Bis = 0
    best_eta_t = eta_t_lower
    best_powers = None
    while eta_t_upper - eta_t_lower > tolerance:
        itr_Bis += 1
        
        eta_t_mid = (eta_t_lower + eta_t_upper) / 2.0
        feasible, powers = check_feasibility(eta_t_mid, A_bar, B_bar, B_tilde, I_M, beta, T, K)
        if feasible:
            best_eta_t = eta_t_mid
            best_powers = powers
            eta_t_lower = eta_t_mid
        else:
            eta_t_upper = eta_t_mid
    print('Bisection iteration:', itr_Bis)
    return best_eta_t, best_powers

# Example usage
max_eta_t, optimal_powers = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, T, K)
print("Maximum eta_t:", max_eta_t)
print("Optimal power allocations:")
for t in range(T):
    print(f"Time slot {t + 1}: {optimal_powers[t]}")

# Compute SINR and latencies
SINR_optimal = compute_SINR(max_eta_t, optimal_powers, A_bar, B_bar, B_tilde, I_M, beta)
latencies_optimal = compute_latencies(SINR_optimal, beta)

# Print latencies
print("\nLatencies:")
#for t in range(T):
for j in range(K):
    print(f" User {j + 1}: {latencies_optimal[t, j]}")


iteration of LP: 1
iteration of LP: 1
iteration of LP: 1
iteration of LP: 1
iteration of LP: 1
iteration of LP: 2
iteration of LP: 1
iteration of LP: 1
iteration of LP: 2
iteration of LP: 1
iteration of LP: 1
iteration of LP: 2
iteration of LP: 1
iteration of LP: 2
iteration of LP: 3
iteration of LP: 4
iteration of LP: 5
iteration of LP: 6
iteration of LP: 7
iteration of LP: 8
iteration of LP: 9
iteration of LP: 10
iteration of LP: 11
iteration of LP: 12
iteration of LP: 13
iteration of LP: 14
iteration of LP: 15
iteration of LP: 16
iteration of LP: 17
iteration of LP: 18
iteration of LP: 19
iteration of LP: 20
iteration of LP: 21
iteration of LP: 22
iteration of LP: 23
iteration of LP: 24
iteration of LP: 25
iteration of LP: 26
iteration of LP: 27
iteration of LP: 28
iteration of LP: 29
iteration of LP: 30
iteration of LP: 31
iteration of LP: 32
iteration of LP: 33
iteration of LP: 34
iteration of LP: 35
iteration of LP: 36
iteration of LP: 37
iteration of LP: 38
iteration of LP: 39
i

C:\Users\afmb\AppData\Local\Temp\ipykernel_12868\2002214089.py:34: RuntimeWarning: overflow encountered in double_scalars
  powers[t, j] = numerator / denominator
C:\Users\afmb\AppData\Local\Temp\ipykernel_12868\2002214089.py:30: RuntimeWarning: invalid value encountered in multiply
  numerator = gamma_t[j] * (np.sum(powers[t] * B_tilde[j]) + I_M[j])


iteration of LP: 99
iteration of LP: 100
iteration of LP: 101
iteration of LP: 102
iteration of LP: 103
iteration of LP: 104
iteration of LP: 105
iteration of LP: 106
iteration of LP: 107
iteration of LP: 108
iteration of LP: 109
iteration of LP: 110
iteration of LP: 111
iteration of LP: 112
iteration of LP: 113
iteration of LP: 114
iteration of LP: 115
iteration of LP: 116
iteration of LP: 117
iteration of LP: 118
iteration of LP: 119
iteration of LP: 120
iteration of LP: 121
iteration of LP: 122
iteration of LP: 123
iteration of LP: 124
iteration of LP: 125
iteration of LP: 126
iteration of LP: 127
iteration of LP: 128
iteration of LP: 129
iteration of LP: 130
iteration of LP: 131
iteration of LP: 132
iteration of LP: 133
iteration of LP: 134
iteration of LP: 135
iteration of LP: 136
iteration of LP: 137
iteration of LP: 138
iteration of LP: 139
iteration of LP: 140
iteration of LP: 141
iteration of LP: 142
iteration of LP: 143
iteration of LP: 144
iteration of LP: 145
iteration of L

iteration of LP: 661
iteration of LP: 662
iteration of LP: 663
iteration of LP: 664
iteration of LP: 665
iteration of LP: 666
iteration of LP: 667
iteration of LP: 668
iteration of LP: 669
iteration of LP: 670
iteration of LP: 671
iteration of LP: 672
iteration of LP: 673
iteration of LP: 674
iteration of LP: 675
iteration of LP: 676
iteration of LP: 677
iteration of LP: 678
iteration of LP: 679
iteration of LP: 680
iteration of LP: 681
iteration of LP: 682
iteration of LP: 683
iteration of LP: 684
iteration of LP: 685
iteration of LP: 686
iteration of LP: 687
iteration of LP: 688
iteration of LP: 689
iteration of LP: 690
iteration of LP: 691
iteration of LP: 692
iteration of LP: 693
iteration of LP: 694
iteration of LP: 695
iteration of LP: 696
iteration of LP: 697
iteration of LP: 698
iteration of LP: 699
iteration of LP: 700
iteration of LP: 701
iteration of LP: 702
iteration of LP: 703
iteration of LP: 704
iteration of LP: 705
iteration of LP: 706
iteration of LP: 707
iteration of 

iteration of LP: 54
iteration of LP: 55
iteration of LP: 56
iteration of LP: 57
iteration of LP: 58
iteration of LP: 59
iteration of LP: 60
iteration of LP: 61
iteration of LP: 62
iteration of LP: 63
iteration of LP: 64
iteration of LP: 65
iteration of LP: 66
iteration of LP: 67
iteration of LP: 68
iteration of LP: 69
iteration of LP: 70
iteration of LP: 71
iteration of LP: 72
iteration of LP: 73
iteration of LP: 74
iteration of LP: 75
iteration of LP: 76
iteration of LP: 77
iteration of LP: 78
iteration of LP: 79
iteration of LP: 80
iteration of LP: 81
iteration of LP: 82
iteration of LP: 83
iteration of LP: 84
iteration of LP: 85
iteration of LP: 86
iteration of LP: 87
iteration of LP: 88
iteration of LP: 89
iteration of LP: 90
iteration of LP: 91
iteration of LP: 92
iteration of LP: 93
iteration of LP: 94
iteration of LP: 95
iteration of LP: 96
iteration of LP: 97
iteration of LP: 98
iteration of LP: 99
iteration of LP: 100
iteration of LP: 101
iteration of LP: 102
iteration of LP: 

iteration of LP: 572
iteration of LP: 573
iteration of LP: 574
iteration of LP: 575
iteration of LP: 576
iteration of LP: 577
iteration of LP: 578
iteration of LP: 579
iteration of LP: 580
iteration of LP: 581
iteration of LP: 582
iteration of LP: 583
iteration of LP: 584
iteration of LP: 585
iteration of LP: 586
iteration of LP: 587
iteration of LP: 588
iteration of LP: 589
iteration of LP: 590
iteration of LP: 591
iteration of LP: 592
iteration of LP: 593
iteration of LP: 594
iteration of LP: 595
iteration of LP: 596
iteration of LP: 597
iteration of LP: 598
iteration of LP: 599
iteration of LP: 600
iteration of LP: 601
iteration of LP: 602
iteration of LP: 603
iteration of LP: 604
iteration of LP: 605
iteration of LP: 606
iteration of LP: 607
iteration of LP: 608
iteration of LP: 609
iteration of LP: 610
iteration of LP: 611
iteration of LP: 612
iteration of LP: 613
iteration of LP: 614
iteration of LP: 615
iteration of LP: 616
iteration of LP: 617
iteration of LP: 618
iteration of 

iteration of LP: 1098
iteration of LP: 1099
iteration of LP: 1100
iteration of LP: 1101
iteration of LP: 1102
iteration of LP: 1103
iteration of LP: 1104
iteration of LP: 1105
iteration of LP: 1106
iteration of LP: 1107
iteration of LP: 1108
iteration of LP: 1109
iteration of LP: 1110
iteration of LP: 1111
iteration of LP: 1112
iteration of LP: 1113
iteration of LP: 1114
iteration of LP: 1115
iteration of LP: 1116
iteration of LP: 1117
iteration of LP: 1118
iteration of LP: 1119
iteration of LP: 1120
iteration of LP: 1121
iteration of LP: 1122
iteration of LP: 1123
iteration of LP: 1124
iteration of LP: 1125
iteration of LP: 1126
iteration of LP: 1127
iteration of LP: 1128
iteration of LP: 1129
iteration of LP: 1130
iteration of LP: 1131
iteration of LP: 1132
iteration of LP: 1133
iteration of LP: 1134
iteration of LP: 1135
iteration of LP: 1136
iteration of LP: 1137
iteration of LP: 1138
iteration of LP: 1139
iteration of LP: 1140
iteration of LP: 1141
iteration of LP: 1142
iteration 

iteration of LP: 1721
iteration of LP: 1722
iteration of LP: 1723
iteration of LP: 1724
iteration of LP: 1725
iteration of LP: 1726
iteration of LP: 1727
iteration of LP: 1728
iteration of LP: 1729
iteration of LP: 1730
iteration of LP: 1731
iteration of LP: 1732
iteration of LP: 1733
iteration of LP: 1734
iteration of LP: 1735
iteration of LP: 1736
iteration of LP: 1737
iteration of LP: 1738
iteration of LP: 1739
iteration of LP: 1740
iteration of LP: 1741
iteration of LP: 1742
iteration of LP: 1743
iteration of LP: 1744
iteration of LP: 1745
iteration of LP: 1746
iteration of LP: 1747
iteration of LP: 1748
iteration of LP: 1749
iteration of LP: 1750
iteration of LP: 1751
iteration of LP: 1752
iteration of LP: 1753
iteration of LP: 1754
iteration of LP: 1755
iteration of LP: 1756
iteration of LP: 1757
iteration of LP: 1758
iteration of LP: 1759
iteration of LP: 1760
iteration of LP: 1761
iteration of LP: 1762
iteration of LP: 1763
iteration of LP: 1764
iteration of LP: 1765
iteration 

iteration of LP: 2207
iteration of LP: 2208
iteration of LP: 2209
iteration of LP: 2210
iteration of LP: 2211
iteration of LP: 2212
iteration of LP: 2213
iteration of LP: 2214
iteration of LP: 2215
iteration of LP: 2216
iteration of LP: 2217
iteration of LP: 2218
iteration of LP: 2219
iteration of LP: 2220
iteration of LP: 2221
iteration of LP: 2222
iteration of LP: 2223
iteration of LP: 2224
iteration of LP: 2225
iteration of LP: 2226
iteration of LP: 2227
iteration of LP: 2228
iteration of LP: 2229
iteration of LP: 2230
iteration of LP: 2231
iteration of LP: 2232
iteration of LP: 2233
iteration of LP: 2234
iteration of LP: 2235
iteration of LP: 2236
iteration of LP: 2237
iteration of LP: 2238
iteration of LP: 2239
iteration of LP: 2240
iteration of LP: 2241
iteration of LP: 2242
iteration of LP: 2243
iteration of LP: 2244
iteration of LP: 2245
iteration of LP: 2246
iteration of LP: 2247
iteration of LP: 2248
iteration of LP: 2249
iteration of LP: 2250
iteration of LP: 2251
iteration 

#The following is without iterations over LP:

In [8]:
import numpy as np
import math
import numpy.linalg as nl
# Define parameters
T = 1  # single time slot for simplicity
K = 5  # number of users
B_tilde = np.array([[0.0, 0.00280774, 0.00227045, 0.00599378, 0.00054239],
                    [0.00280774, 0.0, 0.00090963, 0.10872945, 0.00079498],
                    [0.00227045, 0.00090963, 0.0, 0.00180449, 0.03946987],
                    [0.00599378, 0.10872945, 0.00180449, 0.0, 0.00094696],
                    [0.00054239, 0.00079498, 0.03946987, 0.00094696, 0.0]])
A_bar = 1 * np.array([0.038254, 0.37979634, 6.42646204, 1.4801049, 1.43769989])
B_bar = np.array([0.00206423, 0.04449928, 1.3862424, 0.29103015, 0.29611187])
I_M = np.array([7.78643045e-13, 2.45343970e-12, 1.00922029e-11, 4.84335440e-12, 4.77346915e-12])
beta = np.array([462410*32,462410*32, 462410*32, 462410*32, 462410*32]) #bits 462410*np.ones(K)*32
eps = 0.01

# Function to check feasibility of power allocations and return power values
def check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, beta, T, K):
    itr = 0
    powers = (1/K)*np.ones(K)
    powers_old = 10*(1/K)*np.ones(K)
    for t in range(1):
        gamma_t = 2 ** (eta_t * beta) - 1
        # Solve the system of linear equations for power allocations
        while(itr < 1):
            itr +=1
            powers_old = np.copy(powers)
            for j in range(K):
                numerator = gamma_t[j] * (np.sum(powers[t] * B_tilde[j]) + I_M[j])
                denominator = A_bar[j] - gamma_t[j] * B_bar[j]
                if denominator <= 0:
                    return False, None
                powers[j] = numerator / denominator
        # Check feasibility of the computed powers
        if np.any(powers < 0) or np.any(powers > 1):
            return False, None
    return True, powers

# Function to compute SINR_t^j given powers
def compute_SINR(eta_t, powers, A_bar, B_bar, B_tilde, I_M, beta, K):
    #K = powers.shape
    SINR = np.zeros(K)
    for t in range(1):
        gamma_t = 2 ** (eta_t * beta/(2e6)) - 1
        for j in range(K):
            interference = np.sum(powers * B_tilde[j]) + I_M[j]
            SINR[ j] = A_bar[j] * powers[ j] / (B_bar[j] * powers[ j] + interference)
    return SINR

# Function to compute latencies ell_t^j given SINR_t^j
def compute_latencies(SINR, beta, K):
    #K = SINR.shape
    latencies = np.zeros(K)
    for t in range(1):
        for j in range(K):
            if SINR[ j] > 0:
                latencies[ j] = beta[j] / math.log2(1 + SINR[ j])
            else:
                latencies[ j] = float('inf')  # Handle cases where SINR[t, j] is zero
    return latencies

# Bisection method to find maximum eta_t and store power allocations
def bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, T, K, eta_t_lower=0, eta_t_upper=10.0, tolerance=1e-6):
    itr_Bis = 0
    best_eta_t = eta_t_lower
    best_powers = None
    while eta_t_upper - eta_t_lower > tolerance:
        itr_Bis += 1
        
        eta_t_mid = (eta_t_lower + eta_t_upper) / 2.0
        feasible, powers = check_feasibility(eta_t_mid, A_bar, B_bar, B_tilde, I_M, beta, T, K)
        if feasible:
            best_eta_t = eta_t_mid
            best_powers = powers
            eta_t_lower = eta_t_mid
        else:
            eta_t_upper = eta_t_mid
    print('Bisection iteration:', itr_Bis)
    return best_eta_t, best_powers

# Example usage
max_eta_t, optimal_powers = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, T, K)
print("Maximum eta_t:", max_eta_t)
print(" Optimal power allocations:")
for t in range(T):
    print(f"Powers: {optimal_powers}")

# Compute SINR and latencies
SINR_optimal = compute_SINR(max_eta_t, optimal_powers, A_bar, B_bar, B_tilde, I_M, beta, K)
latencies_optimal = compute_latencies(SINR_optimal, beta, K)

# Print latencies
print("\nLatencies:")
#for t in range(T):
for j in range(K):
    print(f" User {j + 1}: {latencies_optimal[t, j]}")


Bisection iteration: 24
Maximum eta_t: 0
 Optimal power allocations:
Powers: None


C:\Users\afmb\AppData\Local\Temp\ipykernel_18000\713667213.py:24: RuntimeWarning: overflow encountered in power
  gamma_t = 2 ** (eta_t * beta) - 1


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [10]:
num_global_iterations = 15
num_clients= 5
tau_p = min(K,10) #Orthogonal sequences if tau_p=K, else tau_p = 10

tau_c = 200
prelogFactor = (tau_c-tau_p)/(tau_c)
BW = 20e6
B_tau = BW*prelogFactor 

In [11]:

# This is for Lambda1 = sqrt(Aj) and Lambda31 = log2(1 + A/(B + \sum B_tilde + I_M)) 

Bits1 = np.array([[  464797.,   468610.,   465851.,   469044.,   473446.,   484389.,
          498804.,   577234.,   518985.,   522891.,   528936.,   554077.,
          549365.,   539042.,   587185.],
       [  467618.,   478375.,   463619.,   525929.,   536655.,   535322.,
          521062.,   584085.,   644845.,   567841.,   619549.,   497812.,
          571344.,   614093.,   536314.],
       [10712467.,  2916339.,  3820237.,  4054411.,  4155843.,  4130547.,
         4286260.,  4405052.,  4573134.,  4663747.,  4798845.,  5013086.,
         5037204.,  5051960.,  5113774.],
       [  480359.,   519047.,   495766.,   703156.,   612264.,   645155.,
          674419.,   997346.,  1044900.,  1014458.,   964672.,   835123.,
          891481.,   925705.,  1076923.],
       [  473508.,   527138.,   625160.,   665429.,   848236.,   476143.,
          603646.,   817856.,   988325.,   996168.,   919164.,  1088858.,
          952520.,   732544.,   870556.]])

Bits31 = np.array([[ 463123.,  464177.,  462937.,  468672.,  477259.,  462627.,
         472702.,  488884.,  488574.,  488357.,  481475.,  500664.,
         489225.,  471803.,  489969.],
       [ 463774.,  462627.,  464022.,  464828.,  462906.,  469230.,
         471834.,  473198.,  467370.,  468145.,  464797.,  465045.,
         464828.,  465231.,  467091.],
       [9721645., 2493437., 2880162., 2775320., 2951493., 3373248.,
        3752378., 3783564., 3850276., 3945384., 3995852., 4015475.,
        4014948., 4026139., 4060642.],
       [ 517342.,  588766.,  547660.,  642086.,  856389.,  652316.,
         980947., 1217229., 1115735.,  997439., 1003143., 1117595.,
        1080829., 1061826., 1186570.],
       [ 817453., 1077822.,  949575., 1482186., 1780561., 1837043.,
        1895416., 2076084., 2001529., 2222776., 2362059., 2346466.,
        2387634., 2593040., 2650266.]])

In [12]:
for k in range(num_global_iterations):
    print('k:', k)
    max_eta_t, optimal_powers = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, Bits1[:,k]/B_tau, T, K)
    print("Maximum eta_t:", max_eta_t)
   # print(" Optimal power allocations:")

    #print(f"Powers: {optimal_powers}")
    # Compute SINR and latencies
    SINR_optimal = compute_SINR(max_eta_t, optimal_powers, A_bar, B_bar, B_tilde, I_M, Bits1[:,k])
    latencies_optimal = compute_latencies(SINR_optimal, Bits1[:,k])
    print('Latency:',latencies_optimal/B_tau)
    print('max latency:', np.max(latencies_optimal/B_tau))
    print('---Lambda31----')
    max_eta_t3, optimal_powers3 = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, Bits31[:,k]/B_tau, T, K)
    print("Maximum eta_t:", max_eta_t)
    #print(" Optimal power allocations:")
    
    #print(f"Powers: {optimal_powers}")
    # Compute SINR and latencies
    SINR_optimal = compute_SINR(max_eta_t3, optimal_powers3, A_bar, B_bar, B_tilde, I_M, Bits31[:,k])
    latencies_optimal3 = compute_latencies(SINR_optimal, Bits31[:,k])
    print('Latency:',latencies_optimal3/B_tau)
    print('max latency:', np.max(latencies_optimal3/B_tau))
    print('--- For FedAvg ------------')
    max_eta_t_f, optimal_powers_f = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta/B_tau, T, K)
    print("Maximum eta_t_f:", max_eta_t_f)
   # print(" Optimal power allocations:")

    #print(f"Powers: {optimal_powers}")
    # Compute SINR and latencies
    SINR_optimal_f = compute_SINR(max_eta_t_f, optimal_powers_f, A_bar, B_bar, B_tilde, I_M, beta)
    latencies_optimal_f = compute_latencies(SINR_optimal_f, beta)
    print('Latency:',latencies_optimal_f/B_tau)
    print('max latency:', np.max(latencies_optimal_f/B_tau))
    print('-----------------------------------------------------------------------------')

k: 0
Bisection iteration: 24
Maximum eta_t: 4.540690779685974


TypeError: compute_SINR() missing 1 required positional argument: 'K'

In [42]:
# This is for Lambda1 = log2(1 + SNR) not SINR and Lambda31 = lg2(1+SINR) as previous
Bits1 = np.array([[7710241., 2760347., 3429699., 3902542., 4054659., 4469873.,
        4755693., 4854366., 4829721., 4954124., 4985496., 5000500.,
        5123074., 5155655., 5362115.],
       [ 479057.,  498587.,  488667.,  555782.,  637250.,  915196.,
        1005158., 1012319.,  728700., 1120416.,  905214., 1136288.,
        1305796., 1061485., 1311066.],
       [ 465045.,  464518.,  469261.,  467029.,  495022.,  511669.,
         531850.,  563005.,  585821.,  528595.,  590254.,  531788.,
         558789.,  646240.,  560866.],
       [ 467122.,  467246.,  467804.,  463898.,  498866.,  548094.,
         545211.,  476112.,  534547.,  661213.,  608389.,  675380.,
         567283.,  586968.,  606188.],
       [ 463991.,  465634.,  467370.,  464177.,  503020.,  466998.,
         483707.,  511390.,  520194.,  565361.,  585728.,  585201.,
         591928.,  472702.,  544777.]])


Bits31 = np.array([[ 463061.,  465975.,  471617.,  467370.,  477042.,  477321.,
         472237.,  481630.,  473229.,  479491.,  471679.,  482622.,
         477383.,  474624.,  478778.],
       [ 463030.,  463774.,  469354.,  464239.,  475430.,  477817.,
         472082.,  486187.,  463371.,  465572.,  466068.,  463960.,
         464115.,  464053.,  464549.],
       [9460563., 2902389., 3254952., 3842030., 3977841., 4183774.,
        4247417., 4304891., 4438439., 4453815., 4500408., 4547156.,
        4737279., 4705039., 4633057.],
       [ 476174.,  512537.,  776719.,  676155., 1071746., 1053704.,
        1156624., 1217911., 1228296., 1167226., 1270921., 1423317.,
        1361937., 1446257., 1465911.],
       [ 759266., 1041552., 1100948., 1538823., 1502088., 2207400.,
        2468079., 2302446., 2804925., 2418076., 2678104., 2821200.,
        2722775., 2939248., 2992847.]])
#------------------------ The following is with Lambda1 = random, and lambda31 as before:
Bits1 = np.array([[  464518.,   463402.,   472051.,   465665.,   488791.,   470253.,
          471834.,   467618.,   488574.,   472082.,   472392.,   479274.,
          468734.,   481940.,   472578.],
       [  532315.,   561703.,   863829.,  1304277.,  1389031.,  1144999.,
          901773.,  1433702.,  1500445.,  1301239.,  1626770.,  1387202.,
         1736510.,  1727582.,  1824922.],
       [  464177.,   465665.,   472113.,   464518.,   462937.,   475678.,
          522860.,   499052.,   531416.,   501532.,   521651.,   502059.,
          472082.,   516784.,   557828.],
       [  479150.,   486652.,   605971.,   537988.,   708178.,   732544.,
          900533.,   942321.,   847275.,   879453.,  1104947.,   530579.,
          830473.,  1057207.,  1001655.],
       [11920258.,  3603702.,  4128532.,  3970370.,  4464448.,  4678162.,
         4885831.,  5124717.,  5276524.,  5490052.,  5537792.,  5650322.,
         5649857.,  5709780.,  5703084.]])


Bits31 = np.array([[ 462751.,  463898.,  465014.,  483800.,  479987.,  472361.,
         479801.,  493751.,  495766.,  484265.,  476701.,  507360.,
         502617.,  484730.,  488264.],
       [ 464177.,  463123.,  463526.,  468238.,  465138.,  468269.,
         471028.,  471710.,  468858.,  474810.,  471524.,  469013.,
         466316.,  471431.,  483273.],
       [9098762., 2816147., 2705818., 2902420., 2891260., 3091272.,
        3251449., 3608073., 3669763., 3712171., 3791407., 3770389.,
        3846618., 3833536., 3862924.],
       [ 471090.,  570972.,  838037.,  773030.,  967710., 1103242.,
        1159352., 1262520., 1205573., 1219523., 1232016., 1058788.,
        1144720., 1348483., 1307284.],
       [ 907322., 1048155., 1174046., 1454410., 1795162., 2099334.,
        2158730., 2331400., 2391571., 1862618., 2478557., 2631945.,
        2575494., 2766206., 2748691.]])

for k in range(num_global_iterations):
    print('k:', k)
    max_eta_t, optimal_powers = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, Bits1[:,k]/B_tau, T, K)
    print("Maximum eta_t:", max_eta_t)
   # print(" Optimal power allocations:")

    #print(f"Powers: {optimal_powers}")
    # Compute SINR and latencies
    SINR_optimal = compute_SINR(max_eta_t, optimal_powers, A_bar, B_bar, B_tilde, I_M, Bits1[:,k])
    latencies_optimal = compute_latencies(SINR_optimal, Bits1[:,k])
    print('Latency:',latencies_optimal/B_tau)
    print('max latency:', np.max(latencies_optimal/B_tau))
    print('---Lambda31----')
    max_eta_t3, optimal_powers3 = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, Bits31[:,k]/B_tau, T, K)
    print("Maximum eta_t:", max_eta_t)
    #print(" Optimal power allocations:")
    
    #print(f"Powers: {optimal_powers}")
    # Compute SINR and latencies
    SINR_optimal = compute_SINR(max_eta_t3, optimal_powers3, A_bar, B_bar, B_tilde, I_M, Bits31[:,k])
    latencies_optimal3 = compute_latencies(SINR_optimal, Bits31[:,k])
    print('Latency:',latencies_optimal3/B_tau)
    print('max latency:', np.max(latencies_optimal3/B_tau))
    print('--- For FedAvg ------------')
    max_eta_t_f, optimal_powers_f = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta/B_tau, T, K)
    print("Maximum eta_t_f:", max_eta_t_f)
   # print(" Optimal power allocations:")

    #print(f"Powers: {optimal_powers}")
    # Compute SINR and latencies
    SINR_optimal_f = compute_SINR(max_eta_t_f, optimal_powers_f, A_bar, B_bar, B_tilde, I_M, beta)
    latencies_optimal_f = compute_latencies(SINR_optimal_f, beta)
    print('Latency:',latencies_optimal_f/B_tau)
    print('max latency:', np.max(latencies_optimal_f/B_tau))
    print('-----------------------------------------------------------------------------')

k: 0
Bisection iteration: 24
Maximum eta_t: 4.170967936515808
Latency: [[0.06402734 0.01846411 1.10019786 0.47302226 0.2397525 ]]
max latency: 1.1001978555149317
---Lambda31----
Bisection iteration: 24
Maximum eta_t: 4.170967936515808
Latency: [[0.1850061  0.01643797 0.18705683 0.1751588  0.18757273]]
max latency: 0.1875727314327802
--- For FedAvg ------------
Bisection iteration: 24
Maximum eta_t_f: 3.149656057357788
Latency: [[0.56623317 0.58309816 0.31420449 0.31745443 0.31749498]]
max latency: 0.5830981604678843
-----------------------------------------------------------------------------
k: 1
Bisection iteration: 24
Maximum eta_t: 9.999999403953552
Latency: [[0.00801582 0.00918164 0.0101278  0.09010605 0.10000001]]
max latency: 0.10000000596046481
---Lambda31----
Bisection iteration: 24
Maximum eta_t: 9.999999403953552
Latency: [[0.00803521 0.00765288 0.05810003 0.08845272 0.10000001]]
max latency: 0.10000000596046481
--- For FedAvg ------------
Bisection iteration: 24
Maximum eta

C:\Users\afmb\AppData\Local\Temp\ipykernel_12124\1438472338.py:45: RuntimeWarning: overflow encountered in power
  gamma_t = 2 ** (eta_t * beta) - 1


 24
Maximum eta_t: 8.799472451210022
Latency: [[0.03919149 0.0321081  0.49591891 0.12726692 0.11364318]]
max latency: 0.4959189101198891
---Lambda31----
Bisection iteration: 24
Maximum eta_t: 8.799472451210022
Latency: [[0.0094527  0.00795987 0.07924135 0.09031937 0.10000001]]
max latency: 0.10000000596046485
--- For FedAvg ------------
Bisection iteration: 24
Maximum eta_t_f: 3.149656057357788
Latency: [[0.56623317 0.58309816 0.31420449 0.31745443 0.31749498]]
max latency: 0.5830981604678843
-----------------------------------------------------------------------------
k: 13
Bisection iteration: 24
Maximum eta_t: 8.707118034362793
Latency: [[0.03966625 0.0322229  0.4975119  0.12819933 0.11484856]]
max latency: 0.49751190111351884
---Lambda31----
Bisection iteration: 24
Maximum eta_t: 8.707118034362793
Latency: [[0.00927835 0.00812218 0.07898353 0.09093888 0.10000001]]
max latency: 0.10000000596046482
--- For FedAvg ------------
Bisection iteration: 24
Maximum eta_t_f: 3.149656057357788

#The fillowing code is the the SLSQP that solves the optimization problem

In [22]:
import numpy as np
import math
from scipy.optimize import minimize

# Define parameters
T = 1  # single time slot for simplicity
K = 5  # number of users
B_tilde = np.array([[0.0, 0.00280774, 0.00227045, 0.00599378, 0.00054239],
                    [0.00280774, 0.0, 0.00090963, 0.10872945, 0.00079498],
                    [0.00227045, 0.00090963, 0.0, 0.00180449, 0.03946987],
                    [0.00599378, 0.10872945, 0.00180449, 0.0, 0.00094696],
                    [0.00054239, 0.00079498, 0.03946987, 0.00094696, 0.0]])
A_bar = 2 * np.array([0.038254, 0.37979634, 6.42646204, 1.4801049, 1.43769989])
B_bar = np.array([0.00206423, 0.04449928, 1.3862424, 0.29103015, 0.29611187])
I_M = np.array([7.78643045e-13, 2.45343970e-12, 1.00922029e-11, 4.84335440e-12, 4.77346915e-12])
beta = np.array([8, 6, 8, 2, 7])

# Function to check feasibility of power allocations and return power values
def check_feasibility(eta_t, A_bar, B_bar, B_tilde, I_M, beta, T, K):
    powers = (1/K)*np.ones((T, K))
    gamma_t = 2 ** (eta_t * beta) - 1
    for t in range(T):
        # Iteratively solve for power allocations
        for iteration in range(100):  # Arbitrary number of iterations for convergence
            powers_prev = np.copy(powers[t])
            for j in range(K):
                numerator = gamma_t[j] * (np.sum(powers[t] * B_tilde[j]) + I_M[j] - powers[t, j] * B_tilde[j, j])
                denominator = A_bar[j] - gamma_t[j] * B_bar[j]
                if denominator <= 0:
                    return False, None
                powers[t, j] = numerator / denominator
            # Check for convergence
            if np.allclose(powers_prev, powers[t], atol=1e-6):
                break
        # Check feasibility of the computed powers
        if np.any(powers[t] < 0) or np.any(powers[t] > 1):
            return False, None
    return True, powers

# Function to compute SINR_t^j given powers
def compute_SINR(eta_t, powers, A_bar, B_bar, B_tilde, I_M, beta):
    T, K = powers.shape
    SINR = np.zeros((T, K))
    for t in range(T):
        for j in range(K):
            interference = np.sum(powers[t] * B_tilde[j]) + I_M[j] - powers[t, j] * B_tilde[j, j]
            SINR[t, j] = A_bar[j] * powers[t, j] / (B_bar[j] * powers[t, j] + interference)
    return SINR

# Function to compute latencies ell_t^j given SINR_t^j
def compute_latencies(SINR, beta):
    T, K = SINR.shape
    latencies = np.zeros((T, K))
    for t in range(T):
        for j in range(K):
            if SINR[t, j] > 0:
                latencies[t, j] = beta[j] / math.log2(1 + SINR[t, j])
            else:
                latencies[t, j] = float('inf')  # Handle cases where SINR[t, j] is zero
    return latencies

# Bisection method to find maximum eta_t and store power allocations
def bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, T, K, eta_t_lower=0.0, eta_t_upper=10.0, tolerance=1e-6):
    best_eta_t = eta_t_lower
    best_powers = None
    while eta_t_upper - eta_t_lower > tolerance:
        eta_t_mid = (eta_t_lower + eta_t_upper) / 2.0
        feasible, powers = check_feasibility(eta_t_mid, A_bar, B_bar, B_tilde, I_M, beta, T, K)
        if feasible:
            best_eta_t = eta_t_mid
            best_powers = powers
            eta_t_lower = eta_t_mid
        else:
            eta_t_upper = eta_t_mid
    return best_eta_t, best_powers

# Example usage
max_eta_t, optimal_powers = bisection_maximize_eta_t(A_bar, B_bar, B_tilde, I_M, beta, T, K)
print("Maximum eta_t:", max_eta_t)
print("Original Optimal power allocations:")
for t in range(T):
    print(f"Powers: {optimal_powers[t]}")

# Compute SINR and latencies
SINR_optimal = compute_SINR(max_eta_t, optimal_powers, A_bar, B_bar, B_tilde, I_M, beta)
latencies_optimal = compute_latencies(SINR_optimal, beta)

# Print latencies
print("\nLatencies:")
for t in range(T):
    for j in range(K):
        print(f" User {j + 1}: {latencies_optimal[t, j]}")


Maximum eta_t: 0.4186445474624634
Original Optimal power allocations:
Powers: [0.39680664 0.02099984 0.99720833 0.00193908 0.2869322 ]

Latencies:
 User 1: 2.3903869252103953
 User 2: 2.38976912165146
 User 3: 2.3886798307841506
 User 4: 2.388827744114702
 User 5: 2.3886612307775543
